## Challenge: Data cleaning & validation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics
from scipy.stats import ttest_ind

Data Source: WELLCOME Trust between 2012 and 2013, a csv of journals, articles and their costs.
    
Questions: 
1. Determine the five most common journals and the total articles for each. 
2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [2]:
df = pd.read_csv('wellcome.csv')

In [3]:
df.info

<bound method DataFrame.info of                             PMID/PMCID  \
0                                  NaN   
1                           PMC3679557   
2                23043264  PMC3506128    
3                  23438330 PMC3646402   
4                 23438216 PMC3601604    
5                           PMC3579457   
6                           PMC3709265   
7                 23057412 PMC3495574    
8                    PMCID: PMC3780468   
9                    PMCID: PMC3621575   
10                   PMCID: PMC3739413   
11                   PMCID: PMC3530961   
12                   PMCID: PMC3624797   
13                          PMC3413243   
14                          PMC3694353   
15                          PMC3572711   
16                            22610094   
17                   PMCID: PMC3586974   
18        23455506  PMCID: PMC3607399    
19          PMID: 24015914 PMC3833349    
20                       : PMC3805332    
21                                 NaN   
22

In [4]:
df.columns

Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)'],
      dtype='object')

In [5]:
#rename columns
df.rename(columns={'PMID/PMCID':'PMID_PMCID'}, inplace = True) 
df.rename(columns={'Publisher':'publisher'}, inplace = True)
df.rename(columns={'Journal title':'journal_title'}, inplace = True) 
df.rename(columns={'Article title':'Article_title'}, inplace = True) 
df.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'cost'}, inplace = True) 

In [6]:
df.head()

,PMID_PMCID,publisher,journal_title,Article_title,cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [7]:
df.dtypes

PMID_PMCID       object
publisher        object
journal_title    object
Article_title    object
cost             object
dtype: object

In [8]:
df.isnull().sum()/df.count()

PMID_PMCID       0.103216
publisher        0.000000
journal_title    0.000470
Article_title    0.000000
cost             0.000000
dtype: float64

In [9]:
df['journal_title'] = df['journal_title'].replace(np.nan, '', regex=True)

In [10]:
df['journal_title'].isnull().value_counts()

False    2127
Name: journal_title, dtype: int64

### 1. Determine the five most common journals and the total articles for each.

In [11]:
df['journal_title'].nunique()

985

In [12]:
df['journal_title'].value_counts()

PLoS One                                                      92
PLoS ONE                                                      62
Journal of Biological Chemistry                               48
Nucleic Acids Research                                        21
Proceedings of the National Academy of Sciences               19
Human Molecular Genetics                                      18
PLoS Neglected Tropical Diseases                              18
Nature Communications                                         17
PLoS Pathogens                                                15
PLoS Genetics                                                 15
Neuroimage                                                    15
NeuroImage                                                    14
PLOS ONE                                                      14
BMC Public Health                                             14
Brain                                                         14
Movement Disorders       

In [13]:
# Convert all strings to lower case so related titles are counted together.

df['journal_lower'] = df['journal_title'].str.lower()

In [14]:
df['journal_lower'].value_counts()

plos one                                                                            190
journal of biological chemistry                                                      53
neuroimage                                                                           29
plos pathogens                                                                       24
plos genetics                                                                        24
nucleic acids research                                                               23
proceedings of the national academy of sciences                                      20
plos neglected tropical diseases                                                     20
human molecular genetics                                                             19
nature communications                                                                19
brain                                                                                14
bmc public health               

In [15]:
# create DataFrame with short journal title names or acronyms, to check if they coincide with full name titles

short = df[(df['journal_lower'].apply(lambda x: len(x)<7))]
short['journal_lower'].nunique()

33

In [16]:
short['journal_lower'].value_counts()

brain     14
bmj       10
neuron     9
plos 1     7
blood      7
cortex     6
pnas       6
cell       5
pain       4
trials     4
plos       4
bone       3
gut        3
aids       3
gene       3
thorax     2
rna        2
mbio       2
glia       2
age        2
lancet     1
jerph      1
embo       1
chest      1
acta f     1
hernia     1
pntd       1
hpb        1
bjp        1
heart      1
ijtld      1
acta d     1
           1
Name: journal_lower, dtype: int64

In [17]:
# Check whether other top journals have matching acronym entries. (returns empty set)

other_acr = df[(df['journal_lower'].isin(['jbc','nar', 'hmg']))]
other_acr

,PMID_PMCID,publisher,journal_title,Article_title,cost,journal_lower


In [18]:
# Combine all journals related to Plos One.

df['journal_lower'].replace('plosone','plos one', inplace = True)
df['journal_lower'].replace('plos 1', 'plos one', inplace = True)

In [19]:
df['journal_lower'].value_counts()

plos one                                                   206
journal of biological chemistry                             53
neuroimage                                                  29
plos genetics                                               24
plos pathogens                                              24
nucleic acids research                                      23
plos neglected tropical diseases                            20
proceedings of the national academy of sciences             20
human molecular genetics                                    19
nature communications                                       19
bmc public health                                           14
brain                                                       14
movement disorders                                          13
developmental cell                                          12
biochemical journal                                         12
journal of neuroscience                                

In [20]:
df.head()

,PMID_PMCID,publisher,journal_title,Article_title,cost,journal_lower
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,psychological medicine
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,biomacromolecules
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,j med chem
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,j med chem
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88,j org chem


In [21]:
df.cost.dtype

dtype('O')

In [22]:
# Strip out '£' and '$' signs and convert cost to numeric values.

df['cost'] = (df['cost'].str.strip('£'))

In [23]:
for i, cost in enumerate(df.cost):
    if '$' in cost:
        cost = float(cost.replace('$',''))
        cost = cost * 0.71
        df.cost[i] = cost

In [24]:
df.cost = df.cost.apply(pd.to_numeric, errors='coerce')

In [25]:
#  Quick check to see if $ was stripped.  It was.

df.loc[df.journal_title == 'BMC Genomics']

,PMID_PMCID,publisher,journal_title,Article_title,cost,journal_lower
181,PMC2843621,BioMed Central,BMC Genomics,Trichomonas vaginalis vast BspA-like gene fami...,855.1098,bmc genomics
182,PMCID:\n PMC3636053,BioMed Central,BMC Genomics,Enhancing the utility of Proteomics Signature ...,890.7660,bmc genomics
183,3526451,Biomed Central,BMC Genomics,Advances in genome-wide RNAi cellular screens:...,1047.9600,bmc genomics
226,3681581,BioMed Central,BMC Genomics,Transcriptional adaptation of pneumococci and ...,608.5200,bmc genomics


In [26]:
df.cost.dtype

dtype('float64')

In [27]:
df.dtypes

PMID_PMCID        object
publisher         object
journal_title     object
Article_title     object
cost             float64
journal_lower     object
dtype: object

### Here are the five most common journals and the total articles for each.

In [28]:
five = df.groupby(['journal_lower']).count()
five['journal_title'].sort_values(ascending=False).head(5)

journal_lower
plos one                           206
journal of biological chemistry     53
neuroimage                          29
plos pathogens                      24
plos genetics                       24
Name: journal_title, dtype: int64

### 2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [29]:
print(df.cost.max())
print(df.cost.max().dtype)

999999.0
float64


In [30]:
df2 = df[df['cost'] < 15000]  

In [31]:
df2[df2['cost'] > 15000]

,PMID_PMCID,publisher,journal_title,Article_title,cost,journal_lower


In [32]:
topfive = df2.groupby(['journal_lower'])['cost'].agg(['count', 'mean', 'median','std']).reset_index()
topfive_sorted = topfive.sort_values(['count'], ascending=False)
topfive_sorted.head()

,journal_lower,count,mean,median,std
764,plos one,197,935.410964,896.990,195.369314
506,journal of biological chemistry,52,1423.588462,1301.140,411.954360
694,neuroimage,29,2215.168276,2326.430,266.653947
719,nucleic acids research,23,1169.217391,852.000,454.329372
765,plos pathogens,22,1452.277700,1439.225,279.871212
